In [1]:
!pip install aif360
!pip install fairlearn
!pip install scikit-learn pandas numpy matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 2.2 MB/s eta 0:00:00


In [3]:
!mkdir -p data/adult
!wget -O data/adult/adult.data https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget -O data/adult/adult.test https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
!wget -O data/adult/adult.names https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names


--2025-08-07 03:09:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/adult/adult.data’

data/adult/adult.da     [     <=>            ]   3.79M  3.62MB/s    in 1.0s    

2025-08-07 03:09:37 (3.62 MB/s) - ‘data/adult/adult.data’ saved [3974305]

--2025-08-07 03:09:38--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/adult/adult.test’

data/adult/adult.te     [    <=>             ]   1.91M  2.10MB/s    in 0.9s    

2025-08-07 03:09:39 (2.10 MB/s) - ‘da

In [8]:
from google.colab import files
uploaded = files.upload()


Saving adult.zip to adult.zip


In [10]:
import zipfile
import os

zip_path = "adult.zip"  # <-- change this to the uploaded file name
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("adult_data")

os.listdir("adult_data")  # show extracted files


['adult.names', 'old.adult.names', 'adult.data', 'Index', 'adult.test']

In [11]:
import pandas as pd

# Define column names
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Load the dataset
df = pd.read_csv("adult_data/adult.data", header=None, names=columns, skipinitialspace=True)

# Show preview
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Drop rows with missing values
df = df.dropna()

# Encode categorical columns
cat_cols = df.select_dtypes(include=['object']).columns
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

# Split into X and y
X = df.drop('income', axis=1)
y = df['income']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build model
model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7802 - loss: 0.4536 - val_accuracy: 0.8449 - val_loss: 0.3330
Epoch 2/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8417 - loss: 0.3416 - val_accuracy: 0.8545 - val_loss: 0.3242
Epoch 3/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8485 - loss: 0.3275 - val_accuracy: 0.8560 - val_loss: 0.3164
Epoch 4/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8478 - loss: 0.3259 - val_accuracy: 0.8537 - val_loss: 0.3189
Epoch 5/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8486 - loss: 0.3189 - val_accuracy: 0.8580 - val_loss: 0.3142
Epoch 6/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8497 - loss: 0.3173 - val_accuracy: 0.8580 - val_loss: 0.3161
Epoch 7/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8505 - loss: 0.3187 - val_accuracy: 0.8614 - val_loss: 0.3138
Epoch 8/10
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8487 - loss: 0.3206 - val_accuracy: 0.

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


204/204 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8608 - loss: 0.3104
Test Accuracy: 0.85


In [17]:
!pip install aif360


In [18]:
import pandas as pd
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [23]:
import pandas as pd

column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

df = pd.read_csv("adult_data/adult.data", header=None, names=column_names, skipinitialspace=True)
df.dropna(inplace=True)


In [24]:
# Binary encode target and protected attribute
df['income'] = (df['income'].str.strip() == '>50K').astype(int)
df['sex'] = (df['sex'].str.strip() == 'Female').astype(int)

# Save to CSV temporarily (required by AIF360)
df.to_csv("adult_clean.csv", index=False)


In [27]:
df['income'] = (df['income'].astype(str).str.strip() == '>50K').astype(int)
df['sex'] = (df['sex'].astype(str).str.strip() == 'Female').astype(int)


In [28]:
# One-hot encode categorical features
df_encoded = pd.get_dummies(df.drop(columns=['income']), drop_first=True)

# Add label back
df_encoded['income'] = df['income'].values


In [30]:
from aif360.datasets import StandardDataset

dataset = StandardDataset(
    df=df_encoded,
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['sex'],
    privileged_classes=[[0]]  # 0 = Male (privileged)
)


In [37]:
import pandas as pd

column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Reload the full dataset from scratch
df = pd.read_csv("adult_data/adult.data", header=None, names=column_names, skipinitialspace=True)

# Confirm sex values
print(df['sex'].value_counts())  # <- This should show both Male and Female

# Drop missing values safely
df = df.replace('?', pd.NA).dropna()

# Re-encode
df['income'] = (df['income'].astype(str).str.strip() == '>50K').astype(int)
df['sex'] = (df['sex'].astype(str).str.strip() == 'Female').astype(int)

# Confirm females exist with both income levels
print(df[df['sex'] == 1]['income'].value_counts())


sex
Male      21790
Female    10771
Name: count, dtype: int64
income
0    8670
1    1112
Name: count, dtype: int64


In [38]:
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric

# One-hot encode all other features
df_encoded = pd.get_dummies(df.drop(columns=['income']), drop_first=True)
df_encoded['income'] = df['income']

# Create StandardDataset
dataset = StandardDataset(
    df=df_encoded,
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['sex'],
    privileged_classes=[[0]]  # 0 = Male
)

# Compute fairness metrics
metric = BinaryLabelDatasetMetric(
    dataset,
    privileged_groups=[{'sex': 0}],
    unprivileged_groups=[{'sex': 1}]
)

print("📊 Fairness Evaluation:")
print("Statistical Parity Difference:", metric.statistical_parity_difference())
print("Disparate Impact:", metric.disparate_impact())
print("Mean Difference:", metric.mean_difference())


📊 Fairness Evaluation:
Statistical Parity Difference: -0.20015891077100018
Disparate Impact: 0.36222035623624405
Mean Difference: -0.20015891077100018


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Convert df_encoded to features and labels
X = df_encoded.drop(columns=['income']).values
y = df_encoded['income'].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)


In [40]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)


In [41]:
class ResidualGate(nn.Module):
    def __init__(self, input_size, output_size):
        super(ResidualGate, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.gate = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        transformed = self.linear(x)
        gate_weight = self.gate(x)
        return transformed + gate_weight * x[:, :output_size]  # Match shape

class GatedMLP(nn.Module):
    def __init__(self, input_size):
        super(GatedMLP, self).__init__()
        self.layer1 = ResidualGate(input_size, 64)
        self.relu1 = nn.ReLU()
        self.layer2 = ResidualGate(64, 32)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = self.output(x)
        return self.sigmoid(x)


In [42]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=20):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        # Accuracy (optional)
        if epoch % 5 == 0 or epoch == epochs - 1:
            with torch.no_grad():
                model.eval()
                preds = (model(X_test) >= 0.5).float()
                acc = (preds == y_test).float().mean()
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {acc.item():.4f}")
    return model


In [44]:
class ResidualGate(nn.Module):
    def __init__(self, input_size, output_size):
        super(ResidualGate, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.gate = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.Sigmoid()
        )
        self.output_size = output_size  # <-- store this properly!

    def forward(self, x):
        transformed = self.linear(x)
        gate_weight = self.gate(x)
        # Match shape: slice only as many input dims as needed
        residual_input = x[:, :self.output_size]
        return transformed + gate_weight * residual_input


In [45]:
class GatedMLP(nn.Module):
    def __init__(self, input_size):
        super(GatedMLP, self).__init__()
        self.layer1 = ResidualGate(input_size, 64)
        self.relu1 = nn.ReLU()
        self.layer2 = ResidualGate(64, 32)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = self.output(x)
        return self.sigmoid(x)


In [46]:
print("\nTraining GatedMLP (with residual gate):")
gmlp = GatedMLP(X_train.shape[1])
gmlp = train_model(gmlp, X_train, y_train, X_test, y_test)



Training GatedMLP (with residual gate):
Epoch 1/20, Loss: 0.6530, Accuracy: 0.7237
Epoch 6/20, Loss: 0.6019, Accuracy: 0.7477
Epoch 11/20, Loss: 0.5611, Accuracy: 0.7481
Epoch 16/20, Loss: 0.5246, Accuracy: 0.7500
Epoch 20/20, Loss: 0.4965, Accuracy: 0.7534


In [47]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

def to_aif360_dataset(X, y_pred, protected_attr, attr_name='sex'):
    # Create DataFrame for AIF360
    import pandas as pd
    df_aif = pd.DataFrame(X.cpu().numpy())
    df_aif['income'] = y_pred.cpu().numpy().flatten()
    df_aif[attr_name] = protected_attr.cpu().numpy().flatten()  # 1 = Female, 0 = Male

    return BinaryLabelDataset(
        df=df_aif,
        label_names=['income'],
        protected_attribute_names=[attr_name]
    )


In [48]:
# Get predictions for both models
with torch.no_grad():
    y_pred_mlp = (mlp(X_test) >= 0.5).float()
    y_pred_gmlp = (gmlp(X_test) >= 0.5).float()

# Protected attribute (sex column: Female=1, Male=0)
sex_col_idx = df_encoded.columns.get_loc("sex")
sex_test = X_test[:, sex_col_idx].clone()


In [50]:
# Count predicted positives by sex
import pandas as pd

def count_predictions_by_sex(preds, sex_tensor):
    df = pd.DataFrame({
        'sex': sex_tensor.cpu().numpy(),
        'pred': preds.cpu().numpy().flatten()
    })
    return df.groupby('sex')['pred'].value_counts()

print("MLP Prediction counts:")
print(count_predictions_by_sex(y_pred_mlp, sex_test))

print("\nGatedMLP Prediction counts:")
print(count_predictions_by_sex(y_pred_gmlp, sex_test))


MLP Prediction counts:
sex        pred
-0.692518  0.0     4072
 1.444007  0.0     1961
Name: count, dtype: int64

GatedMLP Prediction counts:
sex        pred
-0.692518  0.0     4022
           1.0       50
 1.444007  0.0     1961
Name: count, dtype: int64


In [52]:
# Assuming -0.69 = Male (0), 1.44 = Female (1)
sex_test_binary = (sex_test > 0).int()


In [53]:
# Old:
# sex_test = X_test[:, sex_col_idx]

# New:
sex_test = sex_test_binary  # Now it's 0/1 as required


In [54]:
# Convert predictions to AIF360 datasets
mlp_dataset = to_aif360_dataset(X_test, y_pred_mlp, sex_test)
gmlp_dataset = to_aif360_dataset(X_test, y_pred_gmlp, sex_test)

# Define privileged and unprivileged groups
privileged = [{'sex': 0}]      # Males
unprivileged = [{'sex': 1}]    # Females

# Evaluate fairness
mlp_metric = BinaryLabelDatasetMetric(mlp_dataset, privileged, unprivileged)
gmlp_metric = BinaryLabelDatasetMetric(gmlp_dataset, privileged, unprivileged)

print("📊 Fairness: MLP (No Residual Gate)")
print("Statistical Parity Difference:", mlp_metric.statistical_parity_difference())
print("Disparate Impact:", mlp_metric.disparate_impact())
print("Mean Difference:", mlp_metric.mean_difference())

print("\n📊 Fairness: Gated MLP (With Residual Gate)")
print("Statistical Parity Difference:", gmlp_metric.statistical_parity_difference())
print("Disparate Impact:", gmlp_metric.disparate_impact())
print("Mean Difference:", gmlp_metric.mean_difference())


📊 Fairness: MLP (No Residual Gate)
Statistical Parity Difference: 0.0
Disparate Impact: nan
Mean Difference: 0.0

📊 Fairness: Gated MLP (With Residual Gate)
Statistical Parity Difference: 0.012278978388998035
Disparate Impact: inf
Mean Difference: 0.012278978388998035


/usr/local/lib/python3.11/dist-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
/usr/local/lib/python3.11/dist-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


In [55]:
results = """
Adult Income Dataset - Fairness Comparison

Model: MLP (No Residual Gate)
Accuracy: 0.7464
Statistical Parity Difference: 0.000
Mean Difference: 0.000
Disparate Impact: NaN

Model: Gated MLP (With Residual Gate)
Accuracy: 0.7534
Statistical Parity Difference: 0.0123
Mean Difference: 0.0123
Disparate Impact: inf
"""

# Save to file
with open("fairness_results.txt", "w") as f:
    f.write(results)


In [56]:
from google.colab import files
files.download("fairness_results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from google.colab import files

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define the Gated MLP with a simple residual gate
class GatedMLP(nn.Module):
    def __init__(self, input_size):
        super(GatedMLP, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.gate = nn.Sequential(
            nn.Linear(input_size, 1),
            nn.Sigmoid()
        )
        self.output_activation = nn.Sigmoid()  # <-- Add this

    def forward(self, x):
        transformed = self.linear(x)
        gate_weight = self.gate(x)
        combined = transformed + gate_weight * x[:, :1]
        return self.output_activation(combined)  # <-- Apply sigmoid

# Training function that returns logs
def train_model(model, X_train, y_train, X_test, y_test, epochs=20):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    logs = []

    for epoch in range(1, epochs + 1):
        model.train()
        outputs = model(X_train).squeeze()
        loss = criterion(outputs, y_train.float().squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Evaluate accuracy
        model.eval()
        with torch.no_grad():
            preds = model(X_test).squeeze()
            preds_binary = (preds > 0.5).float()
            acc = (preds_binary == y_test).float().mean().item()

        logs.append([epoch, loss.item(), acc])
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")

    return model, logs

# Convert your data to tensors (replace with actual data!)
# Example: assuming you already have preprocessed X_train, y_train, X_test, y_test
# Replace these with your actual tensors
# X_train = torch.tensor(...)
# y_train = torch.tensor(...)
# X_test = torch.tensor(...)
# y_test = torch.tensor(...)

# Train MLP
print("Training MLP (no residual gate):")
mlp_model, mlp_logs = train_model(MLP(X_train.shape[1]), X_train, y_train, X_test, y_test)

# Train Gated MLP
print("\nTraining GatedMLP (with residual gate):")
gmlp_model, gmlp_logs = train_model(GatedMLP(X_train.shape[1]), X_train, y_train, X_test, y_test)

# Save logs with model names
df_mlp = pd.DataFrame(mlp_logs, columns=["Epoch", "Loss", "Accuracy"])
df_mlp["Model"] = "MLP"

df_gmlp = pd.DataFrame(gmlp_logs, columns=["Epoch", "Loss", "Accuracy"])
df_gmlp["Model"] = "GatedMLP"

# Combine and save
df_all = pd.concat([df_mlp, df_gmlp], ignore_index=True)
df_all.to_csv("training_logs_all.csv", index=False)
files.download("training_logs_all.csv")


Training MLP (no residual gate):
Epoch 1/20, Loss: 0.7265, Accuracy: 0.3562
Epoch 2/20, Loss: 0.7192, Accuracy: 0.3685
Epoch 3/20, Loss: 0.7120, Accuracy: 0.3828
Epoch 4/20, Loss: 0.7050, Accuracy: 0.3991
Epoch 5/20, Loss: 0.6981, Accuracy: 0.4161
Epoch 6/20, Loss: 0.6914, Accuracy: 0.4313
Epoch 7/20, Loss: 0.6848, Accuracy: 0.4430
Epoch 8/20, Loss: 0.6783, Accuracy: 0.4552
Epoch 9/20, Loss: 0.6720, Accuracy: 0.4691
Epoch 10/20, Loss: 0.6657, Accuracy: 0.4796
Epoch 11/20, Loss: 0.6596, Accuracy: 0.4893
Epoch 12/20, Loss: 0.6537, Accuracy: 0.5000
Epoch 13/20, Loss: 0.6478, Accuracy: 0.5075
Epoch 14/20, Loss: 0.6420, Accuracy: 0.5161
Epoch 15/20, Loss: 0.6364, Accuracy: 0.5227
Epoch 16/20, Loss: 0.6308, Accuracy: 0.5298
Epoch 17/20, Loss: 0.6254, Accuracy: 0.5379
Epoch 18/20, Loss: 0.6200, Accuracy: 0.5444
Epoch 19/20, Loss: 0.6147, Accuracy: 0.5504
Epoch 20/20, Loss: 0.6096, Accuracy: 0.5568

Training GatedMLP (with residual gate):
Epoch 1/20, Loss: 0.7233, Accuracy: 0.4559
Epoch 2/20, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>